In [ ]:
from sec_api import EdgarEntitiesApi
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
import asyncio
import websockets


In [ ]:
# Step 2: Load environment variables
load_dotenv('PATH TO YOUR API KEY THAT')

api_key = os.getenv('SEC_API_KEY')
url = "https://api.sec-api.io/insider-trading"
headers = {
    "Authorization": api_key

}



In [ ]:
# Step 1: Import libraries
import requests
import os
from dotenv import load_dotenv

# Step 2: Load environment variables
load_dotenv()

# Step 3: Get the API key
api_key = os.getenv('SEC_API_KEY')

# Step 4: Check if API key was loaded
if not api_key:
    print("Error: SEC_API_KEY not found in environment variables")
    exit()

# Step 5: Set up the request
url = "https://api.sec-api.io/insider-trading"
headers = {
    "Authorization": api_key,
    "Content-Type": "application/json"  # Usually needed for POST requests
}

{
    "query": "issuer.tradingSymbol:TSLA",
    "from": "0",
    "size": "50",
    "sort": [{ "filedAt": { "order": "desc" } }]
  }


In [ ]:
import os, requests

API_KEY = os.getenv("SEC_API_KEY")
url = "https://api.sec-api.io/insider-trading"

payload = {
    "query": "issuer.tradingSymbol:AAPL AND documentType:4",
    "from": 0,
    "size": 10,
    "sort": [{"filedAt": {"order": "desc"}}]
}

headers = {
    "Authorization": API_KEY,
    "Content-Type": "application/json"
}

r = requests.post(url, headers=headers, json=payload, timeout=20)
r.raise_for_status()
data = r.json()
print(len(data.get("transactions", [])), "transactions")


In [ ]:
v = data.get('transactions')

In [ ]:
#Save the JSON file so I can mess around with the formatting and such 
with open("insider_trades.json", "w") as f:
    json.dump(data, f, indent=2) 

In [ ]:
rows = []

with open('insider_trades.json', 'r') as f:
    data = json.load(f)
    for txn in data.get("transactions", []):
        owner = txn.get("reportingOwner", {}).get("name")
        ticker = txn.get("issuer", {}).get("tradingSymbol")
        shares = txn.get("nonDerivativeTable", {}).get("transactions", [])[0].get("amounts", {}).get("shares")

        rows.append({
            "Owner": owner,
            "Ticker": ticker,
            "Shares": shares,
        })


insider_data = pd.DataFrame(rows)
insider_data

In [ ]:
import json, os, pathlib

print("CWD:", os.getcwd())
print("File exists?", pathlib.Path("insider_trades.json").exists())

with open("insider_trades.json", "r") as f:
    payload = json.load(f)

print("Top-level type:", type(payload))
print("Top-level keys (if dict):", list(payload)[:12] if isinstance(payload, dict) else "n/a")
print("transactions count:", len(payload.get("transactions", [])) if isinstance(payload, dict) else "n/a")


In [ ]:
with open("insider_trades.json", "r") as f:
    payload = json.load(f)

for i, filing in enumerate(payload.get("transactions", []), start=1):
    nd = (filing.get("nonDerivativeTable") or {}).get("transactions") or []
    dv = (filing.get("derivativeTable")    or {}).get("transactions") or []
    print(f"Filing {i}: non-deriv legs = {len(nd)}, deriv legs = {len(dv)}")
    if i >= 10:  # just peek at a few
        break


In [ ]:
import json
from flattener import flatten_insider_payload

with open("insider_trades.json","r") as f:
    payload = json.load(f)

df = flatten_insider_payload(payload, debug=True)


df.columns


